<img src="./img/logoconvexbrancomini2.png"  align="right"/>

<!--
<img src="./img/logoconvexbrancomini2.png"  align="right"/>
-->
# Resource Allocation Problem

<!--
<img src="./img/logoboxverde.png" align="right"/>
-->
__by [Daniel Cinalli](http://www.cinalli.com.br)__ - DSc Artificial Intelligence

## Uncapacitated Facility Location - Problem #03



<br/><br/> 
## Notes:

* Coded in Python 3.x
* Using [Anaconda](https://www.anaconda.com/) is recommended
* Run the notebook `online` at [binder](https://mybinder.org/v2/gh/drcinalli/Artificial-Intelligence-and-Transformation/master) [![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/drcinalli/Artificial-Intelligence-and-Transformation/master)
<!-- * [nbviewer](https://nbviewer.jupyter.org/) allows you to switch the notebooks "slides" mode-->

<br> </br>
### Table of Contents

- [Problem](#prob)
- [Simplex](#simplex)
- [Random Heuristic](#random)
- [Lowest Shipping Cost (per Client) Heuristic](#lowShip)
- [Greatest Shipping Cost (per Client) Heuristic](#maxShip)
- [Lowest Shipping & Fixed Costs Heuristic](#lowShipFix)
- [Greatest Shipping & Fixed Costs Heuristic](#maxShipFix)

<br>
<br>

<a id='prob'></a>
## Problem #03

<br>
Facilities <br>
$|I| = 25$ 
<br>
<br>
Clients <br>
$|J| = 50$ 


<br> 
<br>


<a id='simplex'></a>
### Simplex (exact)



In [11]:
from itertools import product
from math import sqrt
import gurobipy as gp
from gurobipy import GRB

# Get Clients and Facilities
def getFacilities_Clients(file_list):
    return int(file_list[0]), int(file_list[1])

# Get Facilities Fixed Costs
def getFacilities_Capacity_FixedCosts(file_list, num_facilities):
    shift = 2
    capacity = []
    cost = []
    
    #loop to get all i location costs
    for i in range(0,num_facilities*2,2):
        capacity.append(int(file_list[i+shift]))
        cost.append(int(file_list[i+1+shift].replace(".","")))
    
    return capacity, cost

# Get Demand and Allocation Costs for j(customer) to each i(client)
def getClient_Demand_AllocationCosts(file_list, num_facilities, num_customers):
    shift = 2 + (num_facilities*2)
    demand = []
    allocation_cost = []
    
    #loop to get all j Clients 
    j=0
    for r in range(0,num_customers):
    
        #get demand
        demand.append(int(file_list[j+shift]))

        #loop to get all i location costs
        for i in range(0,num_facilities):
            allocation_cost.append(float(file_list[j+1+i+shift]))
            
        #fix j
        j += num_facilities+1
 
    
    return demand, allocation_cost

#Read File from OR datasets
fileName='datasets/ORcap104.txt'
ORlist = []

with open(fileName, "r") as f:
    ORlist = f.read().split()
    
##### Sets and Indices #####
num_facilities, num_customers = getFacilities_Clients(ORlist)
capacity, fixed_cost = getFacilities_Capacity_FixedCosts(ORlist, num_facilities)
cartesian_prod = list(product(range(num_customers), range(num_facilities)))
# shipping costs
demand, alloc_cost = getClient_Demand_AllocationCosts(ORlist, num_facilities, num_customers)
shipping_cost = dict(zip(cartesian_prod, alloc_cost))



# MIP  model formulation
m = gp.Model('UFLP')


##### Decision Variable #####
x = m.addVars(num_facilities, vtype=GRB.BINARY, name='x')
y = m.addVars(cartesian_prod, ub=1, vtype=GRB.CONTINUOUS, name='y')

##### Constraints #####
m.addConstrs((y[(c,f)] <= x[f] for c,f in cartesian_prod), name='Shipping')
m.addConstrs((gp.quicksum(y[(c,f)] for f in range(num_facilities)) == 1 for c in range(num_customers)), name='Demand')

##### Objective Function #####
m.setObjective(x.prod(fixed_cost)+y.prod(shipping_cost), GRB.MINIMIZE)

m.optimize()

Gurobi Optimizer version 9.1.0 build v9.1.0rc0 (mac64)
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 1300 rows, 1275 columns and 3750 nonzeros
Model fingerprint: 0x379392c7
Variable types: 1250 continuous, 25 integer (25 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [5e+02, 1e+06]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 3132764.7125
Presolve removed 1116 rows and 1107 columns
Presolve time: 0.02s
Presolved: 184 rows, 168 columns, 454 nonzeros
Found heuristic solution: objective 1090882.1500
Variable types: 152 continuous, 16 integer (16 binary)

Root relaxation: objective 9.289418e+05, 69 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0    928941.75000 928941.750  0.00%     -    

In [12]:
# display optimal values of decision variables

for facility in x.keys():
    if (abs(x[facility].x) > 1e-6):
        print(f"\nBuild a warehouse at location {facility + 1}.")

# Shipments from facilities to customers.

for customer, facility in y.keys():
    if (abs(y[customer, facility].x) > 1e-6):
        print(f"\nClient {customer + 1} receives {round(100*y[customer, facility].x, 2)} % of its demand  from Warehouse {facility + 1} .")

#for v in m.getVars():
#    print(v.varname, v.x)

print(f"\nOptimal total:", m.objVal)

m.write('UFLP_03_Simplex.lp')


Build a warehouse at location 11.

Build a warehouse at location 13.

Build a warehouse at location 18.

Build a warehouse at location 24.

Client 1 receives 100.0 % of its demand  from Warehouse 11 .

Client 2 receives 100.0 % of its demand  from Warehouse 11 .

Client 3 receives 100.0 % of its demand  from Warehouse 11 .

Client 4 receives 100.0 % of its demand  from Warehouse 18 .

Client 5 receives 100.0 % of its demand  from Warehouse 11 .

Client 6 receives 100.0 % of its demand  from Warehouse 11 .

Client 7 receives 100.0 % of its demand  from Warehouse 18 .

Client 8 receives 100.0 % of its demand  from Warehouse 11 .

Client 9 receives 100.0 % of its demand  from Warehouse 11 .

Client 10 receives 100.0 % of its demand  from Warehouse 11 .

Client 11 receives 100.0 % of its demand  from Warehouse 11 .

Client 12 receives 100.0 % of its demand  from Warehouse 11 .

Client 13 receives 100.0 % of its demand  from Warehouse 11 .

Client 14 receives 100.0 % of its demand  from Wa

<br>
<br>

<a id='random'></a>
### Random Heuristic 




In [13]:
#very naive/simple
#for each client, choose randomly one of the Facilities available
import random

result=[]
#choose the Facility for each customer
for i in range(num_customers):
    result.append((i,random.randint(0, num_facilities-1)))
   

#remove duplication of facilities in order to print properly
facs=[]
for i in result:
    facs.append(i[1])
#print(result)
#print("xxx")
#print (facs)
facs=list(set(facs))
    
#print("xxx")
print (facs)

#calculate the setup_cost
totalC=0
for w in facs:
    totalC += fixed_cost[w]
    #print(w)

#print(totalC)

#calculate the shipping cost
for i in result: 
    totalC += shipping_cost.get(i)
    #print (shipping_cost.get(i))
    
print(totalC)



#setup_cost
#cost_per_mile*compute_distance(customers[c], facilities[f]) 
#list(set(output))

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 15, 16, 18, 19, 20, 21, 22, 23, 24]
3804488.6375


In [14]:
for i in facs:
    print(f"\nBuild a warehouse at location {i + 1}.")

for i in result:
    print(f"\nClient {i[0] + 1} receives 100% of its demand  from Warehouse {i[1] + 1} .")

print(f"\nOptimal total:", totalC)


Build a warehouse at location 2.

Build a warehouse at location 3.

Build a warehouse at location 4.

Build a warehouse at location 5.

Build a warehouse at location 6.

Build a warehouse at location 7.

Build a warehouse at location 8.

Build a warehouse at location 9.

Build a warehouse at location 10.

Build a warehouse at location 11.

Build a warehouse at location 12.

Build a warehouse at location 13.

Build a warehouse at location 14.

Build a warehouse at location 16.

Build a warehouse at location 17.

Build a warehouse at location 19.

Build a warehouse at location 20.

Build a warehouse at location 21.

Build a warehouse at location 22.

Build a warehouse at location 23.

Build a warehouse at location 24.

Build a warehouse at location 25.

Client 1 receives 100% of its demand  from Warehouse 17 .

Client 2 receives 100% of its demand  from Warehouse 2 .

Client 3 receives 100% of its demand  from Warehouse 16 .

Client 4 receives 100% of its demand  from Warehouse 12 .

Cl

In [15]:
len(facs)

22

<br>
<br>

<a id='lowShip'></a>
### Lowest Shipping Cost (per client) Heuristic



In [16]:
#Get the lowest shipping cost for each Client


#choose the lowest cost 
path={}
for i in range(num_customers):
    aux={}
    aux_key=()
    for j in range(num_facilities):

        #empty list for the Client
        if not aux:
            aux[(i,j)] = shipping_cost.get((i,j)) 
            aux_key = ((i,j)) 
            #print(aux)
            #print(aux_key)
        elif aux[aux_key]>shipping_cost.get((i,j)):
            #print("....")
            #print (i[0],j[0])
            #print(aux[aux_key])
            #print(shipping_cost.get((i[0],j[0])))
            #print ("  ")
            aux.pop(aux_key)
            aux[(i,j)] = shipping_cost.get((i,j))             
            aux_key = ((i,j))
            
        #print(aux_key)
    #print ("xxxxxxx")
    path.update(aux)

print(path)    

#calculate the setup_cost
facs=[]
for i in path:
    facs.append(i[1])
#print (facs)
facs=list(set(facs))
print (facs)

totalC=0
for w in facs:
    totalC += fixed_cost[w]
    #print(w)

print(totalC)

#calculate the shipping cost
for i in path.values():
    #print(i)
    totalC += i 
    
print(totalC)
print (len(facs))

{(0, 7): 3847.1, (1, 11): 1779.15, (2, 0): 4914.0, (3, 24): 12183.4125, (4, 20): 908.3, (5, 0): 6421.5125, (6, 1): 28499.25, (7, 2): 6370.65, (8, 21): 1050.225, (9, 7): 546.4, (10, 3): 12638.5, (11, 4): 1231.7, (12, 5): 5185.975, (13, 0): 1953.7375, (14, 6): 7310.8125, (15, 7): 8135.7, (16, 3): 2542.5, (17, 8): 11423.1, (18, 20): 2795.65, (19, 6): 5467.3125, (20, 20): 505.4, (21, 9): 11166.8625, (22, 10): 0.0, (23, 0): 4883.0, (24, 11): 5270.65, (25, 4): 2952.9625, (26, 12): 124051.2, (27, 10): 1601.175, (28, 13): 2958.275, (29, 14): 8049.9375, (30, 15): 3285.975, (31, 10): 2398.9, (32, 15): 7346.625, (33, 16): 130895.4, (34, 11): 4891.25, (35, 11): 4154.1, (36, 17): 32993.1125, (37, 18): 10262.7875, (38, 23): 23696.8125, (39, 24): 3636.7, (40, 19): 5043.0, (41, 20): 8098.25, (42, 21): 5775.0, (43, 6): 9331.25, (44, 22): 68042.3625, (45, 23): 21302.8125, (46, 23): 8172.375, (47, 6): 1756.0375, (48, 24): 11949.9, (49, 11): 2614.05}
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 

<br>
<br>

<a id='maxShip'></a>
### Greatest Shipping Cost (per client) Heuristic



In [17]:
#Get the greatest shipping cost for each Client


#choose the lowest cost 
path={}
for i in range(num_customers):
    aux={}
    aux_key=()
    for j in range(num_facilities):

        #empty list for the Client
        if not aux:
            aux[(i,j)] = shipping_cost.get((i,j)) 
            aux_key = ((i,j)) 
            #print(aux)
            #print(aux_key)
        elif not aux[aux_key]>shipping_cost.get((i,j)):
            #print("....")
            #print (i[0],j[0])
            #print(aux[aux_key])
            #print(shipping_cost.get((i[0],j[0])))
            #print ("  ")
            aux.pop(aux_key)
            aux[(i,j)] = shipping_cost.get((i,j))             
            aux_key = ((i,j))
            
        #print(aux_key)
    #print ("xxxxxxx")
    path.update(aux)

print(path)    

#calculate the setup_cost
facs=[]
for i in path:
    facs.append(i[1])
#print (facs)
facs=list(set(facs))
print (facs)

totalC=0
for w in facs:
    totalC += fixed_cost[w]
    #print(w)

print(totalC)

#calculate the shipping cost
for i in path.values():
    #print(i)
    totalC += i 
    
print(totalC)
print (len(facs))

{(0, 23): 11656.275, (1, 23): 6919.7625, (2, 23): 66973.2, (3, 22): 142841.7375, (4, 14): 2512.9375, (5, 23): 53852.6625, (6, 22): 266358.375, (7, 22): 108940.8375, (8, 14): 2869.7625, (9, 14): 2314.8, (10, 22): 477721.5625, (11, 22): 84557.9, (12, 22): 139911.375, (13, 23): 15041.8125, (14, 23): 54681.1875, (15, 14): 39867.75, (16, 22): 17709.925, (17, 22): 282712.3, (18, 22): 22071.0875, (19, 23): 16314.1875, (20, 22): 3108.875, (21, 23): 77038.2375, (22, 22): 50189.2125, (23, 23): 32615.4, (24, 22): 64804.575, (25, 22): 33148.1625, (26, 1): 406770.0, (27, 22): 53033.5125, (28, 23): 43386.025, (29, 23): 57636.5625, (30, 23): 23235.7125, (31, 22): 30199.575, (32, 23): 69570.3125, (33, 22): 1415639.4, (34, 23): 28075.9375, (35, 22): 27353.925, (36, 22): 394035.9625, (37, 22): 218284.7875, (38, 14): 73399.3125, (39, 22): 35444.5, (40, 22): 143032.0875, (41, 22): 93199.6875, (42, 14): 23103.4375, (43, 23): 47856.25, (44, 14): 224408.1375, (45, 14): 75966.2875, (46, 14): 21287.025, (47, 1

<a id='lowShipFix'></a>
### Lowest Shipping & Fixed Costs Heuristic



In [18]:
#Get the lowest shipping cost for each Client


#choose the lowest cost 
path={}
for i in range(num_customers):
    aux={}
    aux_key=()
    for j in range(num_facilities):

        #empty list for the Client
        if not aux:
            aux[(i,j)] = shipping_cost.get((i,j)) + fixed_cost[j]
            aux_key = ((i,j)) 
            #print(aux)
            #print(aux_key)
        elif aux[aux_key]>(shipping_cost.get((i,j)) + fixed_cost[j]):
            #print("....")
            #print (i[0],j[0])
            #print(aux[aux_key])
            #print(shipping_cost.get((i[0],j[0])))
            #print ("  ")
            aux.pop(aux_key)
            aux[(i,j)] = shipping_cost.get((i,j))+ fixed_cost[j]             
            aux_key = ((i,j))
            
        #print(aux_key)
    #print ("xxxxxxx")
    path.update(aux)

print(path)    

#calculate the setup_cost
facs=[]
for i in path:
    facs.append(i[1])
#print (facs)
facs=list(set(facs))
print (facs)

totalC=0
for w in facs:
    totalC += fixed_cost[w]
    #print(w)

print(totalC)

#calculate the shipping cost
for i in path.values():
    #print(i)
    totalC += i 
    
print(totalC)
print (len(facs))

{(0, 10): 5219.5, (1, 10): 2296.8, (2, 10): 9828.0, (3, 10): 23096.675, (4, 10): 1133.05, (5, 10): 7029.425, (6, 1): 53499.25, (7, 10): 12741.3, (8, 10): 1395.9, (9, 10): 928.8, (10, 10): 25277.0, (11, 10): 2463.4, (12, 10): 10371.95, (13, 10): 2434.575, (14, 10): 14621.625, (15, 10): 16271.4, (16, 10): 3062.3, (17, 10): 22846.2, (18, 10): 4123.9, (19, 10): 7312.5, (20, 10): 780.9, (21, 10): 22333.725, (22, 10): 0.0, (23, 10): 7106.0, (24, 10): 10541.3, (25, 10): 3268.9, (26, 12): 149051.2, (27, 10): 1601.175, (28, 10): 5916.55, (29, 10): 16099.875, (30, 10): 4440.975, (31, 10): 2398.9, (32, 10): 14693.25, (33, 16): 155895.4, (34, 10): 6353.75, (35, 10): 5984.1, (36, 17): 57993.1125, (37, 10): 20525.575, (38, 10): 42775.875, (39, 10): 5658.0, (40, 10): 10086.0, (41, 10): 16196.5, (42, 10): 11550.0, (43, 10): 14550.0, (44, 22): 93042.3625, (45, 10): 42605.625, (46, 10): 11183.25, (47, 10): 2174.375, (48, 10): 23899.8, (49, 10): 4001.55}
[1, 10, 12, 16, 17, 22]
125000
1113661.5750000002


<a id='maxShipFix'></a>
### Greatest Shipping & Fixed Costs Heuristic



In [19]:
#Get the lowest shipping cost for each Client


#choose the lowest cost 
path={}
for i in range(num_customers):
    aux={}
    aux_key=()
    for j in range(num_facilities):

        #empty list for the Client
        if not aux:
            aux[(i,j)] = shipping_cost.get((i,j)) + fixed_cost[j]
            aux_key = ((i,j)) 
            #print(aux)
            #print(aux_key)
        elif not aux[aux_key]>(shipping_cost.get((i,j)) + fixed_cost[j]):
            #print("....")
            #print (i[0],j[0])
            #print(aux[aux_key])
            #print(shipping_cost.get((i[0],j[0])))
            #print ("  ")
            aux.pop(aux_key)
            aux[(i,j)] = shipping_cost.get((i,j))+ fixed_cost[j]             
            aux_key = ((i,j))
            
        #print(aux_key)
    #print ("xxxxxxx")
    path.update(aux)

print(path)    

#calculate the setup_cost
facs=[]
for i in path:
    facs.append(i[1])
#print (facs)
facs=list(set(facs))
print (facs)

totalC=0
for w in facs:
    totalC += fixed_cost[w]
    #print(w)

print(totalC)

#calculate the shipping cost
for i in path.values():
    #print(i)
    totalC += i 
    
print(totalC)
print (len(facs))

{(0, 23): 36656.275, (1, 23): 31919.7625, (2, 23): 91973.2, (3, 22): 167841.7375, (4, 14): 27512.9375, (5, 23): 78852.6625, (6, 22): 291358.375, (7, 22): 133940.8375, (8, 14): 27869.7625, (9, 14): 27314.8, (10, 22): 502721.5625, (11, 22): 109557.9, (12, 22): 164911.375, (13, 23): 40041.8125, (14, 23): 79681.1875, (15, 14): 64867.75, (16, 22): 42709.925, (17, 22): 307712.3, (18, 22): 47071.0875, (19, 23): 41314.1875, (20, 22): 28108.875, (21, 23): 102038.2375, (22, 22): 75189.2125, (23, 23): 57615.4, (24, 22): 89804.575, (25, 22): 58148.1625, (26, 1): 431770.0, (27, 22): 78033.5125, (28, 23): 68386.025, (29, 23): 82636.5625, (30, 23): 48235.7125, (31, 22): 55199.575, (32, 23): 94570.3125, (33, 22): 1440639.4, (34, 23): 53075.9375, (35, 22): 52353.925, (36, 22): 419035.9625, (37, 22): 243284.7875, (38, 14): 98399.3125, (39, 22): 60444.5, (40, 22): 168032.0875, (41, 22): 118199.6875, (42, 14): 48103.4375, (43, 23): 72856.25, (44, 14): 249408.1375, (45, 14): 100966.2875, (46, 14): 46287.02